In [ ]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 119.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

(…)nstruct-sharded/resolve/main/config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)ded/resolve/main/configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


(…)-sharded/resolve/main/modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(…)rded/resolve/main/generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

(…)arded/resolve/main/tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

(…)ruct-sharded/resolve/main/tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

(…)ded/resolve/main/special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = load_dataset("json", data_files="/content/drive/MyDrive/chatdoctor5k.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-1cdef0b1c9bd94b6/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input'],
        num_rows: 5452
    })
})

In [ ]:
data["train"][0]

{'output': "Well, based on what you're telling me, it sounds like you may be suffering from panic disorder. The best course of action is to start with psychotherapy and mental health counseling. Additionally, we should conduct an electrocardiogram to make sure that there are no physical issues causing your panic attacks. We will also need to perform a depression screen and a toxicology screen to rule out any other underlying causes. Finally, I would recommend a comprehensive psychological and psychiatric evaluation and therapy to help manage your symptoms.",
 'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.",
 'input': "Doctor, I have been experiencing sudden and frequent panic attacks. I don't know what to do."}

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["input"]}
<assistant>: {data_point["output"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/5452 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['output', 'instruction', 'input', 'input_ids', 'attention_mask'],
    num_rows: 5452
})

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.238700
2,2.136300
3,2.159900
4,2.505500
5,2.237500
6,2.046800
7,2.421000
8,2.406100
9,2.372500
10,2.144300


TrainOutput(global_step=1363, training_loss=1.2668938603107316, metrics={'train_runtime': 8186.158, 'train_samples_per_second': 0.666, 'train_steps_per_second': 0.167, 'total_flos': 9694044789222144.0, 'train_loss': 1.2668938603107316, 'epoch': 1.0})

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "/content/model"

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>:I woke up today and i am feeling feverish.I also vomitted twice this morning.I also have a headache.What should i do?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>:I woke up today and i am feeling feverish.I also vomitted twice this morning.I also have a headache.What should i do?
<assistant>: Based on your symptoms, it seems like you have a viral infection. We will need to run some tests to confirm the diagnosis. We will start with a complete physical skin exam to check for any skin lesions. We will also need to perform a hematologic test to check your blood counts. Additionally, we will need to do a radiographic imaging procedure to check for any abnormalities in your brain. We will also need to perform a urinalysis to check for any signs of infection in your urine.
CPU times: user 38 s, sys: 33.2 ms, total: 38 s
Wall time: 41.3 s


In [ ]:
%%time
device = "cuda:0"
prompt = """
<human>:I have been experiencing a skin pigmentation disorder lately. My skin has developed dark patches and discolored spots. What can I do to treat this?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>:I have been experiencing a skin pigmentation disorder lately. My skin has developed dark patches and discolored spots. What can I do to treat this?
<assistant>: Based on your symptoms, I would recommend a combination of medications. You will need to take Clobetasol Topical, Fluocinonide Topical, and Betamethasone Topical to reduce the dark patches and discolored spots. You will also need to use Clotrimazole Topical, Fluorouracil Topical, and Salicylic Acid Topical to help with the skin pigmentation disorder. Additionally, you will need to use a sunscreen and wear protective clothing to prevent further damage to your skin. If the condition is severe, you may also need to take oral medications such as Clobazam, Clonazepam, or Valproic Acid. If you have any other medical conditions, we will need to address those as well. Finally, we will also need to monitor your blood pressure and kidney function. Is there anything else I can help you with?
User 
CPU times: user 1min 12s, sys: 91

In [ ]:
%%time
device = "cuda:0"
prompt = """
<human>: I woke up today and I am not feeling well.What should i do?
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: I woke up today and I am not feeling well.What should i do?
<assistant>: You should take some medication to help with your symptoms. You should take a combination of Acetaminophen, Hydrocodone, and Oxycodone to manage your pain and discomfort. You should also take a combination of Chlorzoxazone and Diclofenac to help with your muscle pain and inflammation. Additionally, you should take a combination of Chlorzoxazone and Diclofenac to help with your joint pain. You should also take a combination of Chlorzoxazone and Diclofenac to help with your joint pain. Finally, you should take a combination of Chlorzoxazone and Diclofenac to help with your joint pain. You should also take a combination of Chlorzoxazone and Diclofenac to help with your joint pain. Finally, you should take a combination of Chlorzoxazone and Diclofenac to help with your joint pain. You should also take a combination of Chlorz
CPU times: user 1min 19s, sys: 91.1 ms, total: 1min 19s
Wall time: 1min 20s


In [ ]:
%%time
device = "cuda:0"
prompt = """
<human>:
<assistant>: I had 3 teeth pulled on one side in a row they used 2 stetches to hold the gums together. where one of the stetches was there was a big whole and lots of pain where it was in the gum line area. This pain has been there at that same spot for 10 days now and is not lighting up any....is that normal?
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>:
<assistant>: I had 3 teeth pulled on one side in a row they used 2 stetches to hold the gums together. where one of the stetches was there was a big whole and lots of pain where it was in the gum line area. This pain has been there at that same spot for 10 days now and is not lighting up any....is that normal?
<assistant>: Yes, it is normal to experience pain after tooth extraction. However, if the pain is severe and not improving after a few days, it may be a sign of infection. I would recommend that you follow the post-operative care instructions and come in for a follow-up appointment to ensure proper healing. If the pain persists, we may need to perform further tests to determine the cause of the infection. We may also need to perform a biopsy to check for any damage to the surrounding tissue. In the meantime, we will provide you with pain medication and antibiotics to help manage the pain and prevent any infection. We will also monitor your vital signs and perform a compl